In [1]:
import pandas as pd
import torch
import os
import numpy as np
import tensorflow as tf

In [2]:
def load_data_from_csv(file_path):
    df = pd.read_csv(file_path)
    labels = df[df.columns[-1]]
    features = df.drop(df.columns[-1], axis=1).values
    labels = np.where(labels, 1, 0)

    # X = torch.tensor(features, dtype=torch.float32)
    # y = torch.tensor(labels, dtype=torch.int64)

    return features, labels

In [3]:
from google.colab import drive, files
import os

drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/Colab Notebooks/mobile-proj/data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
filepaths = []
for filename in os.listdir(data_path):
    filepath = os.path.join(data_path, filename)
    if os.path.isfile(filepath) and filepath[-3:] =="csv":
        filepaths.append(filepath)



In [5]:
X_list = []
y_list = []
for file in filepaths:
    X, y = load_data_from_csv(file)
    X_list.append(X)
    y_list.append(y)

X = np.concatenate(X_list, axis=0)
y = np.concatenate(y_list, axis=0)

In [6]:
print(X.shape)
print(y.shape)

(623, 99)
(623,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
X_train.shape

(498, 99)

In [9]:
from tensorflow.keras import layers, models

class PostureDetectionModel(tf.keras.Model):
    def __init__(self):
        super(PostureDetectionModel, self).__init__()

        self.fc1 = layers.Dense(64, activation='relu', input_shape=(99,))
        self.dropout1 = layers.Dropout(0.5)
        self.fc2 = layers.Dense(32, activation='relu')
        self.dropout2 = layers.Dropout(0.5)
        self.fc3 = layers.Dense(32, activation='relu')
        self.dropout3 = layers.Dropout(0.5)
        self.fc4 = layers.Dense(1, activation='sigmoid')

    def call(self, x):
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

model = PostureDetectionModel()

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
from torch.utils.data import TensorDataset, DataLoader

model.fit(X_train, y_train, epochs=25, batch_size=5, shuffle=True, validation_split=0.2)

Epoch 1/25
80/80 [==============================] - 3s 9ms/step - loss: 0.8822 - accuracy: 0.5503 - val_loss: 0.6436 - val_accuracy: 0.7600
Epoch 2/25
80/80 [==============================] - 0s 5ms/step - loss: 0.6809 - accuracy: 0.6106 - val_loss: 0.6000 - val_accuracy: 0.8100
Epoch 3/25
80/80 [==============================] - 0s 5ms/step - loss: 0.6368 - accuracy: 0.6281 - val_loss: 0.5472 - val_accuracy: 0.8300
Epoch 4/25
80/80 [==============================] - 0s 5ms/step - loss: 0.5569 - accuracy: 0.7111 - val_loss: 0.4844 - val_accuracy: 0.8300
Epoch 5/25
80/80 [==============================] - 0s 4ms/step - loss: 0.5090 - accuracy: 0.7513 - val_loss: 0.4424 - val_accuracy: 0.8300
Epoch 6/25
80/80 [==============================] - 0s 5ms/step - loss: 0.4939 - accuracy: 0.7739 - val_loss: 0.4324 - val_accuracy: 0.8100
Epoch 7/25
80/80 [==============================] - 0s 5ms/step - loss: 0.4757 - accuracy: 0.7638 - val_loss: 0.4288 - val_accuracy: 0.8100
Epoch 8/25
80/80 [==

In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=5)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

25/25 [==============================] - 0s 2ms/step - loss: 0.1441 - accuracy: 0.9680
Test Accuracy: 96.80%


In [13]:
model.summary()


Model: "posture_detection_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  6400      
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  2080      
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  1056      
                                                                 
 dropout_2 (Dropout)         multiple                  0         
                                                                 
 dense_3 (Dense)             multiple      

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

def representative_data_gen():
    for input_value, _ in dataset.batch(1).take(100):
        input_value = tf.cast(input_value, tf.float32)
        yield [input_value]

converter.representative_dataset = representative_data_gen
converter.optimizations = [tf.lite.Optimize.DEFAULT]

lite_quant_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [16]:
model_save_path = "/content/drive/MyDrive/Colab Notebooks/mobile-proj"
file_path = os.path.join(model_save_path, 'quantized_model.tflite')

with open(file_path, 'wb') as f:
    f.write(lite_quant_model)


In [17]:
interpreter = tf.lite.Interpreter(model_path=file_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

correct_predictions = 0
total_predictions = 0

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

for x, y in test_dataset:
    x = tf.cast(x, dtype=tf.float32)
    x = tf.expand_dims(x, axis=0)
    interpreter.set_tensor(input_details[0]['index'], x)

    interpreter.invoke()

    y_pred = interpreter.get_tensor(output_details[0]['index'])

    if round(y_pred[0][0]) == y.numpy():
        correct_predictions += 1
    total_predictions += 1

accuracy = correct_predictions / total_predictions
print(f'Quantized Model Accuracy: {accuracy * 100:.2f}%')

Quantized Model Accuracy: 96.80%
